In [ ]:
%pip install matplotlib

In [ ]:
%pip install tensorflow

In [ ]:
%pip install scikeras

In [ ]:
import os
import math, numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from scikeras.wrappers import KerasClassifier
import joblib

In [ ]:
DATA_DIR = Path('dataset')

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
img_size = (224, 224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset", validation_split=0.2, subset="training", seed=123,
    image_size=img_size, batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset", validation_split=0.2, subset="validation", seed=123,
    image_size=img_size, batch_size=batch_size
)
class_names = train_ds.class_names

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

train_ds = (train_ds
    .shuffle(1000)
    .map(lambda x,y: (preprocess_input(tf.cast(data_augmentation(x, training=True), tf.float32)), y),
         num_parallel_calls=AUTOTUNE)
    .cache().prefetch(AUTOTUNE)
)
val_ds = (val_ds
    .map(lambda x,y: (preprocess_input(tf.cast(x, tf.float32)), y),
         num_parallel_calls=AUTOTUNE)
    .cache().prefetch(AUTOTUNE)
)

# --- sanity check: should be in [-1, 1] ---
for xb, yb in train_ds.take(1):
    print("range:", float(tf.reduce_min(xb)), float(tf.reduce_max(xb)))  # expect ~[-1, 1]

# --- model (NO preprocess_input here) ---
def build_model(num_classes):
    base = MobileNetV2(input_shape=(*img_size, 3), include_top=False, weights="imagenet")
    base.trainable = False

    inputs = layers.Input(shape=(*img_size, 3))
    x = base(inputs, training=False)        # inputs already preprocessed
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

model = build_model(len(class_names))

history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
base_model.trainable = True
for layer in model.get_layer("mobilenetv2_1.00_224").layers[:100]:  # freeze first 100 layers
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(train_ds, validation_data=val_ds, epochs=5)

In [ ]:
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

import matplotlib.pyplot as plt
epochs_range = range(len(acc))
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.axvline(x=len(history.history['accuracy'])-1, linestyle='--', label='Fine-tune start')
plt.legend(loc='lower right'); plt.title('Accuracy over Epochs')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.axvline(x=len(history.history['accuracy'])-1, linestyle='--')
plt.legend(loc='upper right'); plt.title('Loss over Epochs')
plt.show()

In [ ]:
model.save("animal_classifier.h5")